In [12]:
import pandas as pd

In [13]:
crimes = pd.read_csv('data/crime.csv')
education = pd.read_csv('data/education.csv')
population = pd.read_csv('data/population.csv')
visitors = pd.read_csv('data/visitors.csv')

### Cleaning and Combining the 3 Datasets

In [14]:
# Removing the year 2021 as this is not included in the other datasets
crimes = crimes.loc[crimes['REF_DATE'] != 2021]

# Remove unwanted columns. I kept the 'GEO', 'REF_DATE', and 'VALUE' columns.
crimes = crimes[['GEO', 'REF_DATE', 'VALUE']]

# Group by 'GEO' and 'REF_DATE'. Here I summed together the values for each province for each year.
crimes_grouped = crimes.groupby(['GEO', 'REF_DATE'])['VALUE'].sum().reset_index()

# Clean 'GEO' names. Remove the number in brackets at the end of each province name.
crimes_grouped['GEO'] = crimes_grouped['GEO'].str.replace('\s*\[\d+\]', '', regex=True)

# Filter out unwanted rows. I removed the rows that contained the following value in the 'GEO' column.
crimes_grouped = crimes_grouped.loc[crimes_grouped['GEO'] != 'Canadian Forces Military Police']

# Rename columns. I renamed the columns to 'Province', 'Year', and 'Crimes'.
crimes_grouped = crimes_grouped.rename(columns={'VALUE': 'Crimes', 'GEO': 'Province', 'REF_DATE': 'Year'})

# Group Territories. I grouped the territories together and summed the values for each year.
territories = ['Northwest Territories', 'Nunavut', 'Yukon']
total_territories_crimes = crimes_grouped[crimes_grouped['Province'].isin(territories)].groupby('Year')['Crimes'].sum().reset_index()
total_territories_crimes['Province'] = 'Territories'

# Remove Territories from crimes_grouped and add total_territories_crimes to crimes_grouped.
crimes_grouped = pd.concat([crimes_grouped, total_territories_crimes], ignore_index=True)
crimes_grouped = crimes_grouped[~crimes_grouped['Province'].isin(territories)]

# Group Territories for the Population dataset
total_territories_population = population[population['Province'].isin(territories)].groupby('Year')['Population'].sum().reset_index()
total_territories_population['Province'] = 'Territories'

# Remove individual Territories from the Population dataset and add the grouped data
population = pd.concat([population, total_territories_population], ignore_index=True)
population = population[~population['Province'].isin(territories)]

# Group Territories for the Education dataset
total_territories_visitors = visitors[visitors['Province'].isin(territories)].groupby('Year')['Visitors'].sum().reset_index()
total_territories_visitors['Province'] = 'Territories'

# Remove individual Territories from the Education dataset and add the grouped data
visitors = pd.concat([visitors, total_territories_visitors], ignore_index=True)
visitors = visitors[~visitors['Province'].isin(territories)]

# Merging crimes_grouped with education based on Province and Year
merged_df = crimes_grouped.merge(education, on=['Province', 'Year'], how='outer')

# Merging the previous result with the population dataset based on Province and Year
final_df = merged_df.merge(population, on=['Province', 'Year'], how='outer')

final_df_with_visitors = final_df.merge(visitors, on=['Province', 'Year'], how='outer')

In [15]:
final_df_with_visitors

,Province,Year,Crimes,Registrants,Population,Visitors
0,Alberta,2017,1005325.43,194010,4258195,2215382
1,Alberta,2018,1043221.45,200391,4317665,2385856
2,Alberta,2019,1071487.39,203823,4384848,1988895
3,Alberta,2020,933926.17,203523,4421857,378321
4,British Columbia,2017,838288.24,286914,4956550,8759579
5,British Columbia,2018,852692.63,295320,5040353,9376652
6,British Columbia,2019,949793.36,297432,5130780,8760281
7,British Columbia,2020,871957.23,287523,5151920,1677587
8,Manitoba,2017,361011.29,62802,1340565,448291
9,Manitoba,2018,361049.00,63363,1357968,496719
